# 🚗 Model 2: Dynamic Pricing Based on Parking Demand


## 🎯 Objective
This notebook calculates parking demand based on various features like occupancy, queue length, traffic condition, and vehicle type. Then it calculates a final price and visualizes the results using Bokeh.


### 🧪 Importing Required Libraries  
I import libraries like pandas for handling data, numpy for calculations, and Bokeh for creating interactive visualizations within the notebook.


In [22]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
output_notebook()

### 🔧 Loading the Dataset  
I load the dataset which contains real-time parking information including capacity, vehicle type, queue length, traffic level, and more.


In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/SakshiSan/dynamic-pricing-parking/main/dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


### 🚦 Traffic Condition Mapping  
I convert traffic levels from text to numeric form to use them in our formula. Low = 1, Average = 2, High = 3.


In [24]:
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

### 🚗 Vehicle Type Weighting  
I assign a numerical weight to each type of vehicle to estimate how much impact it has on demand. For example, trucks affect more than cycles.


In [25]:
vehicle_weight_map = {'car': 2, 'bike': 1, 'truck': 3, 'cycle': 0.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weight_map)

### 📊 Calculating Raw Demand  
I use a linear formula that combines multiple features to estimate the current demand at the parking lot.


In [26]:
df['Demand'] = (
    0.4 * (df['Occupancy'] / df['Capacity']) +
    0.3 * df['QueueLength'] -
    0.2 * df['TrafficLevel'] +
    0.2 * df['IsSpecialDay'] +
    0.3 * df['VehicleWeight']
)

### 🔄 Normalizing Demand  
To keep pricing stable, I normalize the demand score between 0 and 1.


In [27]:
df['Demand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())

### 💰 Calculating Final Price  
I calculate the final price using the normalized demand. As demand increases, the price increases smoothly.


In [28]:
base_price = 10
lambda_factor = 1
df['FinalPrice'] = base_price * (1 + 0.5 * df['Demand'])

### 📈 Visualizing Final Price Over Time (Using Bokeh)  
I use Bokeh to create an interactive graph showing how price changes based on demand across different time intervals.


In [29]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool

output_notebook()

p = figure(title="Final Price Over Time",
           x_axis_label='Record Index',
           y_axis_label='Final Price',
           width=800,
           height=400,
           tools="pan,wheel_zoom,box_zoom,reset,save")

hover = HoverTool(tooltips=[("Index", "$index"), ("Price", "@y")])
p.add_tools(hover)

p.line(x=df.index, y=df['FinalPrice'], legend_label="Final Price", line_width=2, color="navy")

show(p)


### ✅ Conclusion  
I successfully implemented a dynamic pricing system using a demand function based on real-world features like occupancy, traffic, and vehicle type. The final prices adapt smoothly and are visualized effectively using Bokeh.
